In [1]:
import pymysql
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import transformations
import config
%matplotlib inline

In [2]:
conn = pymysql.connect(config.host, user=config.username,port=config.port,
                           passwd=config.password)

#gather all historical data to build model
RideWaits = pd.read_sql_query("call DisneyDB.RideWaitQuery('2,7,8,9')", conn)


starter_data = RideWaits.copy()

#transform data for model bulding
RideWaits = transformations.transformData(RideWaits)

C:\Users\chrisA\Documents\DisneyWaitTimes\DisneyWaits\src\transformations.py:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  RideWaits["Month"] = RideWaits["Date"].dt.month
C:\Users\chrisA\Documents\DisneyWaitTimes\DisneyWaits\src\transformations.py:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  RideWaits["TimeSinceRideOpen"] = (RideWaits["Date"] - RideWaits["OpeningDate"]).dt.days
C:\Users\chrisA\Documents\DisneyWaitTimes\DisneyWaits\src\transformations.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [3]:
def create_dummies(df,column_name):
    dummies = pd.get_dummies(df[column_name],prefix=column_name)
    df = pd.concat([df,dummies],axis=1)
    df = df.drop([column_name], axis = 1)
    return df

def get_shift(day, steps):
    previous_steps = {}
    for i in range(1,1+steps):
        current_steps = []
        test_day_current = day.reset_index()
        for index,row in test_day_current.iterrows():
            if index in list(range(i)):
                current_steps.append(0)
            else:
                current_steps.append(test_day_current.loc[index - i,'Wait'])
        
        name = "previous_step"+str(i)
        previous_steps[name] = current_steps
    
    for key,value in previous_steps.items():
        day[key] = value
        
    return day
    
    
def shift_data(ride_data, shift_range):
    new_data_frame = pd.DataFrame()
    distinct_rides = list(ride_data['RideId'].unique())
    for ride in distinct_rides:
        this_ride = ride_data[ride_data['RideId'] == ride]
        day_list = list(this_ride['Date'].unique())
        for day in day_list:
            day_data = this_ride[this_ride['Date'] == day]
            new_data = get_shift(day_data, shift_range)
            new_data_frame = pd.concat([new_data_frame, new_data])

    return new_data_frame

def model_transformation(data, num_shifts, start_day = True, by_ride = True):
    ride_waits = data
    
    if by_ride:
        important_columns = ['Wait','DayOfWeek','Weekend','inEMH','EMHDay','MagicHourType','Month','TimeSinceOpen','TimeSinceMidday','MinutesSinceOpen']
        dummy_columns = ['DayOfWeek','Weekend','inEMH','EMHDay','MagicHourType','Month']
    else:
        important_columns = ['RideId','Date', 'Wait','Name','Tier','Location','IntellectualProp','ParkId','DayOfWeek','Weekend','CharacterExperience','inEMH','EMHDay','TimeSinceOpen','TimeSinceMidday','MagicHourType','MinutesSinceOpen','Month']
        ride_waits = ride_waits[ride_waits['Location'] != ""]
        dummy_columns = ['RideId','Tier','Location','IntellectualProp','ParkId','DayOfWeek','Weekend','CharacterExperience','inEMH','EMHDay','MagicHourType','Month']
        ride_waits = ride_waits.drop(['Name'], axis = 1)
    ride_waits = ride_waits[important_columns]
    ride_waits = ride_waits.dropna(how = "any")
    
    if start_day == False:
        ride_waits = shift_data(ride_waits,num_shifts)
    
    for column in dummy_columns:
        ride_waits = create_dummies(ride_waits, column)
        
    correlation = ride_waits.corr()['Wait']
    key_correlations = correlation[abs(correlation) > .005]
    important_cols = list(key_correlations.index)
    shift_columns = []
    if start_day == False:
        shift_columns = ["previous_step" + str(x+1)  for x in range(num_shifts)]
    important_cols = important_cols + ["Wait","MinutesSinceOpen"] + shift_columns
    important_cols = [x for x in important_cols if x != "Weekend_0"]
    important_cols = list(set(important_cols))
    ride_waits_key = ride_waits[important_cols]
    
    
    return ride_waits_key



def new_data_transform(data, num_shifts, important_cols, start_day = True, by_ride = True):
    ride_waits = data
    
    if by_ride:
        important_columns = ['Wait','DayOfWeek','Weekend','inEMH','EMHDay','MagicHourType','Month','TimeSinceOpen','TimeSinceMidday','MinutesSinceOpen']
        dummy_columns = ['DayOfWeek','Weekend','inEMH','EMHDay','MagicHourType','Month']
    else:
        important_columns = ['RideId','Date', 'Wait','Name','Tier','Location','IntellectualProp','ParkId','DayOfWeek','Weekend','CharacterExperience','inEMH','EMHDay','TimeSinceOpen','TimeSinceMidday','MagicHourType','MinutesSinceOpen','Month']
        ride_waits = ride_waits[ride_waits['Location'] != ""]
        dummy_columns = ['RideId','Tier','Location','IntellectualProp','ParkId','DayOfWeek','Weekend','CharacterExperience','inEMH','EMHDay','MagicHourType','Month']
        ride_waits = ride_waits.drop(['Name'], axis = 1)
    

    ride_waits = ride_waits[important_columns]
    
    ride_waits = ride_waits.dropna(how = "any")
    
    if start_day == False:
        ride_waits = shift_data(ride_waits,num_shifts)
    for column in dummy_columns:
        ride_waits = create_dummies(ride_waits, column)
        
    missing_cols = [x for x in important_cols if x not in ride_waits.columns]
    for col in missing_cols:
        ride_waits[col] = 0
        
    return ride_waits

In [4]:
from datetime import datetime
from pytz import timezone
tz = timezone('US/Eastern')
dtime = datetime.now(tz)
dtime = dtime.replace(hour = 7,minute = 0, second = 0, microsecond = 0)
date = dtime.date()
time = dtime.time().strftime("%H:%M")
from datetime import datetime
from dateutil.relativedelta import relativedelta

def date_range(start_date, end_date, increment, period):
    result = []
    nxt = start_date
    delta = relativedelta(**{period:increment})
    while nxt <= end_date:
        result.append(nxt)
        nxt += delta
    return result

end_time = dtime.replace(hour = 23, minute = 45, second = 0, microsecond = 0)
time_list = date_range(dtime, end_time, 15, 'minutes')
time_list = [x.time().strftime("%H:%M") for x in time_list]


park_hours = pd.read_sql_query("select * from DisneyDB.ParkHours where Date = '" + str(date) + "'", conn)
todays_hours = park_hours

In [5]:
def get_conf_interval(clf, df):
    conf_high_list = []
    conf_low_list = []
    for index, row in df.iterrows():
        current_row = df.loc[[index]]
        all_predictions = [estimator.predict(current_row) for estimator in clf.estimators_]
        mean = np.mean(all_predictions)
        pred_std = np.std(all_predictions)
        conf_high = (mean + 2*pred_std)
        conf_low = (mean - 2*pred_std)
        conf_high_list.append(conf_high)
        conf_low_list.append(conf_low)
    
    df['confidence_high'] = conf_high_list
    df['confidence_low'] = conf_low_list
    return df

In [6]:
def make_daily_prediction(current_ride,ride, time_list, best_params, todays_predictions,todays_hours):
    ride_predictions = {}
    current_ride_fm = current_ride.copy()
    current_ride_fm = transformations.transformData(current_ride_fm)
    #print(current_ride.shape[0])
    #print(current_ride.columns)
    model_data = model_transformation(current_ride_fm, 1)
    important_columns = [x for x in model_data.columns if x != "Wait"]
    clf = RandomForestRegressor(**best_params)
    #scores = cross_val_score(clf, model_data[important_columns],model_data['Wait'], scoring = "neg_median_absolute_error", cv = 3)
    #ride_score = scores.mean()
    #ride_predictions['score'] = ride_score
    #print(model_data.head())
    clf.fit(model_data[important_columns], model_data['Wait'])
    predictions_frame = pd.DataFrame()
    ride_starter = current_ride.iloc[[0]]
    
    predictions_frame = pd.concat([ride_starter]*len(time_list),ignore_index = True)
    #need to change the park hours and date
    current_park_id = predictions_frame['ParkId'][0]
    todays_hours = park_hours[park_hours['ParkId'] == current_park_id]
    todays_date = todays_hours['Date'].iloc[0]
    park_open = todays_hours['ParkOpen'].iloc[0]
    park_close = todays_hours['ParkClose'].iloc[0]
    emh_open = todays_hours['EMHOpen'].iloc[0]
    emh_close = todays_hours['EMHClose'].iloc[0]

    predictions_frame['Date'] = todays_date
    predictions_frame['EMHOpen'] = emh_open
    predictions_frame['ParkOpen'] = park_open
    predictions_frame['ParkClose'] = park_close
    predictions_frame['EMHClose'] = emh_close


    predictions_frame['Time'] = time_list
    predictions_frame = transformations.transformData(predictions_frame)
    # print(predictions_frame)
    #predictions_frame = transformations.transformData(predictions_frame)
    model_predictions_frame = new_data_transform(predictions_frame, 3, important_columns)
    predictions_frame['predicted_wait'] = clf.predict(model_predictions_frame[important_columns])
    model_predictions_frame = get_conf_interval(clf,model_predictions_frame[important_columns])
    predictions_frame['confidence_high'] = model_predictions_frame['confidence_high']
    predictions_frame['confidence_low'] = model_predictions_frame['confidence_low']
    
    ride_predictions['predictions'] = predictions_frame
    todays_predictions[ride] = ride_predictions

In [7]:
%%time
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
best_params = {'criterion': 'mse',
 'max_depth': 10,
 'max_features': 'auto',
 'min_samples_leaf': 5,
 'min_samples_split': 2,
 'n_estimators': 100}

rides = list(set(starter_data['Name']))
#rides = ["Seven Dwarfs Mine Train"]
global todays_predictions
todays_predictions = {}

import threading
num_threads = len(rides)

threads = []
for i in range(num_threads):
    print(rides[i-1])
    ride = rides[i-1]
    current_ride = starter_data.copy()
    current_ride = starter_data[current_ride['Name'] == ride]
    process = threading.Thread(target = make_daily_prediction, args = [current_ride,ride,time_list, best_params, todays_predictions, todays_hours])
    process.start()
    threads.append(process)

for process in threads:
    process.join()


Spaceship Earth
Big Thunder Mountain Railroad
Its Tough to be a Bug!
Jungle Cruise
Toy Story Mania!
Avatar Flight of Passage
Meet Favorite Disney Pals at Adventurers Outpost
Walt Disneys Carousel of Progress
Kilimanjaro Safaris


C:\Users\chrisA\Documents\DisneyWaitTimes\DisneyWaits\src\transformations.py:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  RideWaits["Month"] = RideWaits["Date"].dt.month
C:\Users\chrisA\Documents\DisneyWaitTimes\DisneyWaits\src\transformations.py:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  RideWaits["TimeSinceRideOpen"] = (RideWaits["Date"] - RideWaits["OpeningDate"]).dt.days
C:\Users\chrisA\Documents\DisneyWaitTimes\DisneyWaits\src\transformations.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

Mickey and Minnie Starring in Red Carpet Dreams


C:\Users\chrisA\Documents\DisneyWaitTimes\DisneyWaits\src\transformations.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  RideWaits["TimeSinceRideOpen"] = (RideWaits["Date"] - RideWaits["OpeningDate"]).dt.days


Star Wars Launch Bay Theater
The Magic Carpets of Aladdin
Alien Swirling Saucers
Celebrity Spotlight
DINOSAUR
Soarin


...


The Hall of Presidents


C:\Users\chrisA\Documents\DisneyWaitTimes\DisneyWaits\src\transformations.py:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  RideWaits["DayOfWeek"] = pd.Categorical(RideWaits["DayOfWeek"])
C:\Users\chrisA\Documents\DisneyWaitTimes\DisneyWaits\src\transformations.py:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  RideWaits["DayOfWeek"] = pd.Categorical(RideWaits["DayOfWeek"])


Mission: SPACE


C:\Users\chrisA\Documents\DisneyWaitTimes\DisneyWaits\src\transformations.py:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  RideWaits["Month"] = RideWaits["Date"].dt.month
C:\Users\chrisA\Documents\DisneyWaitTimes\DisneyWaits\src\transformations.py:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  RideWaits["TimeSinceRideOpen"] = (RideWaits["Date"] - RideWaits["OpeningDate"]).dt.days
C:\Users\chrisA\Documents\DisneyWaitTimes\DisneyWaits\src\transformations.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

Wall time: 2min 33s


In [8]:
metric = []
for key, value in todays_predictions.items():
    print(str(key) + " : " + str(value['predictions'].shape[0]))
    

Walt Disneys Carousel of Progress : 40
Star Wars Launch Bay Theater : 44
A Pirates Adventure ~ Treasures of the Seven Seas : 40
Tom Sawyer Island : 40
Walt Disneys Enchanted Tiki Room : 40
Royal Sommerhus : 52
Swiss Family Treehouse : 40
Jungle Cruise : 40
The Magic Carpets of Aladdin : 40
Big Thunder Mountain Railroad : 40
Dumbo the Flying Elephant : 40
Buzz Lightyears Space Ranger Spin : 40
Space Mountain : 40
The Barnstormer : 40
Meet Mickey Mouse at Town Square Theater : 40
Splash Mountain : 40
Monsters, Inc. Laugh Floor : 40
Pirates of the Caribbean : 40
Prince Charming Regal Carrousel : 40
Under the Sea ~ Journey of The Little Mermaid : 40
Celebrity Spotlight : 44
The Many Adventures of Winnie the Pooh : 40
Astro Orbiter : 40
Alien Swirling Saucers : 44
Mickeys PhilharMagic : 40
Peter Pans Flight : 40
Meet Favorite Disney Pals at Adventurers Outpost : 44
Mickey and Minnie Starring in Red Carpet Dreams : 44
DINOSAUR : 44
Meet Rapunzel and Tiana at Princess Fairytale Hall : 40
Meet

In [9]:
all_predictions = pd.DataFrame()
for key,value in todays_predictions.items():
    current_frame = value['predictions']
    current_frame = current_frame[['RideId','Time','predicted_wait','confidence_high','confidence_low']]
    current_frame.columns = ['RideId','Time','PredictedWait','ConfidenceHigh','ConfidenceLow']
    current_frame['PredictedWait'] = [int(str(x).split(".")[0]) for x in current_frame['PredictedWait']]
    current_frame['ConfidenceHigh'] = [int(str(x).split(".")[0]) for x  in current_frame['ConfidenceHigh']]
    current_frame['ConfidenceLow'] = [int(str(x).split(".")[0]) for x in current_frame['ConfidenceLow']]
    all_predictions = pd.concat([all_predictions, current_frame])

In [10]:
RideIds = list(set(RideWaits['RideId']))


In [11]:
rides_not_predicted = [x for x in RideIds if x not in list(set(all_predictions["RideId"]))]

In [12]:
rides_not_predicted

[207395, 387133, 18361662, 80010193, 80069754]

In [16]:
if len(rides_not_predicted) > 0:
    best_params = {'criterion': 'mse',
     'max_depth': 10,
     'max_features': 'auto',
     'min_samples_leaf': 5,
     'min_samples_split': 2,
     'n_estimators': 100}

    #rides = list(set(starter_data['Name']))
    rides = rides_not_predicted
    todays_predictions = {}

    import threading
    num_threads = len(rides)

    threads = []
    for i in range(num_threads):
        print(rides[i-1])
        ride = rides[i-1]
        current_ride = starter_data.copy()
        current_ride = starter_data[current_ride['RideId'] == ride]
        process = threading.Thread(target = make_daily_prediction, args = [current_ride,ride,time_list, best_params, todays_predictions, todays_hours])
        process.start()
        threads.append(process)

    for process in threads:
        process.join()
        
    more_predictions = pd.DataFrame()
    for key,value in todays_predictions.items():
        current_frame = value['predictions']
        current_frame = current_frame[['RideId','Time','predicted_wait','confidence_high','confidence_low']]
        current_frame.columns = ['RideId','Time','PredictedWait','ConfidenceHigh','ConfidenceLow']
        current_frame['PredictedWait'] = [int(str(x).split(".")[0]) for x in current_frame['PredictedWait']]
        current_frame['ConfidenceHigh'] = [int(str(x).split(".")[0]) for x  in current_frame['ConfidenceHigh']]
        current_frame['ConfidenceLow'] = [int(str(x).split(".")[0]) for x in current_frame['ConfidenceLow']]
        more_predictions = pd.concat([more_predictions, current_frame])
    
    
    all_predictions = pd.concat([all_predictions, more_predictions])

207395


In [17]:
rides_not_predicted = [x for x in RideIds if x not in list(set(all_predictions["RideId"]))]
rides_not_predicted

[]

In [18]:
conn = pymysql.connect(config.host, user=config.username,port=config.port,
                           passwd=config.password)
cur = conn.cursor()
query = "delete from DisneyDB.Ride_Waits_Today_Predicted where RideId > 0"
cur.execute(query)
conn.commit()

for index,row in all_predictions.iterrows():
    query = "insert into DisneyDB.Ride_Waits_Today_Predicted (RideId, Time, PredictedWait, ConfidenceHigh, ConfidenceLow) values (%i, '%s', %i, %i, %i)" %(row['RideId'], row['Time'], row['PredictedWait'], row['ConfidenceHigh'],row['ConfidenceLow'])
    cur.execute(query)
    conn.commit()